In [ ]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.4/413.4 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.6/230.6 kB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 9.3 MB/s eta 0:00:00


In [ ]:
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 7.1 MB/s eta 0:00:00


In [ ]:
!pip install torch

In [ ]:
!pip install transformers

In [ ]:
!pip install transformers datasets evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.8 MB/s eta 0:00:00


## login ke hugging

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

## import dataset

In [ ]:
from datasets import load_dataset, Audio

minds = load_dataset("PolyAI/minds14", name="en-US", split="train")

ConnectionError: ignored

In [ ]:
minds = minds.train_test_split(test_size=0.2) #bagi dataset buat test

In [ ]:
minds #cek data

In [ ]:
minds = minds.remove_columns(["path", "transcription", "english_transcription", "lang_id"]) # apus kolom tidak perlu

In [ ]:
minds["train"][0]

In [ ]:
labels = minds["train"].features["intent_class"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i) # membuat dict untuk maping label ke id
    id2label[str(i)] = label # membuat dict untuk maping id ke label

NameError: ignored

In [ ]:
id2label[str(2)]

'app_error'

## Preprocess

In [ ]:
from transformers import AutoFeatureExtractor

feature_extractor = AutoFeatureExtractor.from_pretrained("facebook/wav2vec2-base")

preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.84k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:381: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [ ]:
minds = minds.cast_column("audio", Audio(sampling_rate=16_000)) #resample ke 16000 khz
minds["train"][0]

{'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/a19fbc5032eacf25eab0097832db7b7f022b42104fbad6bd5765527704a428b9/en-US~PAY_BILL/602bad1fbb1e6d0fbce921e3.wav',
  'array': array([2.27500306e-04, 1.41795375e-04, 1.59611227e-05, ...,
         3.03630880e-03, 1.73226779e-03, 5.63404872e-04]),
  'sampling_rate': 16000},
 'intent_class': 13}

In [ ]:
def preprocess_function(examples):
    audio_arrays = [x["array"] for x in examples["audio"]]
    inputs = feature_extractor(
        audio_arrays, sampling_rate=feature_extractor.sampling_rate, max_length=16000, truncation=True
    )
    return inputs

In [ ]:
encoded_minds = minds.map(preprocess_function, remove_columns="audio", batched=True)
encoded_minds = encoded_minds.rename_column("intent_class", "label")

Map:   0%|          | 0/450 [00:00<?, ? examples/s]

Map:   0%|          | 0/113 [00:00<?, ? examples/s]

In [ ]:
import evaluate

accuracy = evaluate.load("accuracy")

In [ ]:
import numpy as np


def compute_metrics(eval_pred):
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=eval_pred.label_ids)

In [ ]:
from transformers import AutoModelForAudioClassification, TrainingArguments, Trainer

num_labels = len(id2label)
model = AutoModelForAudioClassification.from_pretrained(
    "facebook/wav2vec2-base", num_labels=num_labels, label2id=label2id, id2label=id2label
)

Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['classifier.weight', 'classifier.bias', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'projector.weight', 'projector.bias', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import optuna

def objective(trial):
    # Tentukan hyperparameter yang akan di-tune
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 5e-3)
    num_train_epochs = trial.suggest_int('num_train_epochs', 2, 10)
    per_device_train_batch_size = trial.suggest_categorical('per_device_train_batch_size', [8, 16, 32])
    gradient_accumulation_steps = trial.suggest_categorical('gradient_accumulation_steps', [1, 2, 4])

    # Inisialisasi model dengan hyperparameter yang di-tune
    model = AutoModelForAudioClassification.from_pretrained(
        "facebook/wav2vec2-base",
        num_labels=num_labels,
        label2id=label2id,
        id2label=id2label,
    )

    # Definisi argumen pelatihan
    training_args = TrainingArguments(
        output_dir="latihan_speech_classification",
        evaluation_strategy="epoch",
        save_strategy="epoch",
        learning_rate=learning_rate,
        per_device_train_batch_size=per_device_train_batch_size,
        gradient_accumulation_steps=gradient_accumulation_steps,
        per_device_eval_batch_size=per_device_train_batch_size,
        num_train_epochs=num_train_epochs,
        warmup_ratio=0.1,
        logging_steps=10,
        load_best_model_at_end=True,
        metric_for_best_model="accuracy",
        push_to_hub=True
        )

    # Inisialisasi Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=encoded_minds["train"],  # Pastikan Anda telah menyiapkan data terlebih dahulu
        eval_dataset=encoded_minds["test"],    # Sesuaikan dengan nama dataset evaluasi Anda
        tokenizer=feature_extractor,
        compute_metrics=compute_metrics,        # Sesuaikan dengan fungsi komputasi metrik Anda
    )

    # Lakukan pelatihan
    trainer.train()

    # Hitung dan kembalikan metrik evaluasi (bisa diganti dengan metrik lainnya)
    result = trainer.evaluate()
    accuracy = result['eval_accuracy']
    return accuracy

In [ ]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=10)

# Cetak hyperparameter terbaik
best_params = study.best_params
print("Best Hyperparameters:", best_params)

[I 2023-12-12 09:34:44,329] A new study created in memory with name: no-name-2afa52d2-31ad-4eb4-a393-4a437128c81d
/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:381: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['classifier.weight', 'projector.weight', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'classifier.bias', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'projector.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
0,No log,2.739328,0.035398
1,No log,2.661594,0.035398


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


[I 2023-12-12 09:35:45,265] Trial 0 finished with value: 0.035398230088495575 and parameters: {'learning_rate': 0.0017405789120064192, 'num_train_epochs': 2, 'per_device_train_batch_size': 32, 'gradient_accumulation_steps': 4}. Best is trial 0 with value: 0.035398230088495575.
/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:381: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['classifier.weight', 'projector.weight', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'classifier.bias', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations

Epoch,Training Loss,Validation Loss,Accuracy
0,No log,2.669502,0.035398
2,2.667300,2.645062,0.070796
4,2.654900,2.653293,0.035398
6,2.633400,2.653797,0.079646
8,2.637400,2.653886,0.079646


...

[I 2023-12-12 09:59:56,035] Trial 8 finished with value: 0.07964601769911504 and parameters: {'learning_rate': 0.0024491528214864557, 'num_train_epochs': 5, 'per_device_train_batch_size': 32, 'gradient_accumulation_steps': 4}. Best is trial 4 with value: 0.11504424778761062.
/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:381: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['classifier.weight', 'projector.weight', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'classifier.bias', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.w

Epoch,Training Loss,Validation Loss,Accuracy
0,No log,2.739184,0.035398
1,2.679700,2.643738,0.079646
2,2.670000,2.657755,0.070796
4,2.649200,2.641687,0.035398
5,2.641500,2.641354,0.079646
6,2.635500,2.643207,0.079646
7,2.635500,2.645498,0.079646


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.

[I 2023-12-12 10:03:14,947] Trial 9 finished with value: 0.07964601769911504 and parameters: {'learning_rate': 0.0018879612460626829, 'num_train_epochs': 8, 'per_device_train_batch_size': 16, 'gradient_accumulation_steps': 4}. Best is trial 4 with value: 0.11504424778761062.


Best Hyperparameters: {'learning_rate': 0.0034193556521543325, 'num_train_epochs': 8, 'per_device_train_batch_size': 32, 'gradient_accumulation_steps': 2}


## buat model dengan best param

In [ ]:
best_learning_rate = 0.0034193556521543325
best_num_train_epochs = best_params['num_train_epochs']
best_per_device_train_batch_size = best_params['per_device_train_batch_size']
best_gradient_accumulation_steps = best_params['gradient_accumulation_steps']

training_args = TrainingArguments(
    output_dir="latihan_speech_classification",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=best_learning_rate,
    per_device_train_batch_size=best_per_device_train_batch_size,
    gradient_accumulation_steps=best_gradient_accumulation_steps,
    per_device_eval_batch_size=best_per_device_train_batch_size,
    num_train_epochs=best_num_train_epochs,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_minds["train"],
    eval_dataset=encoded_minds["test"],
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics,
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
0,No log,2.663228,0.070796
2,2.674900,2.677411,0.053097
4,2.670300,2.642891,0.053097
6,2.647000,2.654476,0.053097
8,2.640800,2.642282,0.061947
10,2.647700,2.642577,0.053097
12,2.641600,2.640993,0.053097
14,2.637900,2.643412,0.070796
16,2.640100,2.638076,0.053097
18,2.632400,2.645837,0.053097


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.

TrainOutput(global_step=210, training_loss=2.643123844691685, metrics={'train_runtime': 675.2274, 'train_samples_per_second': 19.993, 'train_steps_per_second': 0.311, 'total_flos': 1.143945016704e+17, 'train_loss': 2.643123844691685, 'epoch': 28.0})

In [ ]:
trainer.push_to_hub()

model.safetensors:   0%|          | 0.00/378M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

events.out.tfevents.1702375525.b70713547168.5181.23:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

'https://huggingface.co/AptaArkana/latihan_speech_classification/tree/main/'